In [1]:
from app.configuration.config import TinvestSettings
from datetime import datetime, timezone, timedelta

from tinkoff.invest import CandleInterval
from app.services.tinvest import TinvestAdapter

In [2]:
config = TinvestSettings(_env_file='../app/.env')
print(config.model_dump())
tinvest = TinvestAdapter(config)

{'token': 't.8DpIsag8_t2bHcaPEXZiAxDLdxbyqP7MXvDwoamPBWSDBD7dgQeMNutgas5Ay83YOlLsA-m8qSPm8Sz-FMaNuw', 'account_id': 'fc66cf9b-8fb8-4d9e-ba79-a5e8b87c5aa7'}


In [3]:
symbol = tinvest.get_instrument(ticker="SBER", class_code="TQBR")

end_date_utc = datetime.now(timezone.utc)
days_count = 10
start_date_utc = end_date_utc - timedelta(days=days_count)

candles = tinvest.get_candles(
    instrument_id=symbol.uid,
    start_date_utc=start_date_utc,
    end_date_utc=end_date_utc,
    interval=CandleInterval.CANDLE_INTERVAL_10_MIN)


In [4]:
candles

,time_utc,open,high,low,close,volume,is_complete,candle_source
0,2025-01-24 16:10:00+00:00,281.22,281.48,281.08,281.34,17410,True,1
1,2025-01-24 16:20:00+00:00,281.34,281.48,281.30,281.35,9153,True,1
2,2025-01-24 16:30:00+00:00,281.37,281.40,280.97,281.10,16139,True,1
3,2025-01-24 16:40:00+00:00,281.13,281.13,280.81,280.84,8287,True,1
4,2025-01-24 16:50:00+00:00,280.83,281.04,280.76,281.04,6811,True,1
...,...,...,...,...,...,...,...,...
602,2025-02-03 15:20:00+00:00,278.81,279.10,278.65,279.08,38005,True,1
603,2025-02-03 15:30:00+00:00,279.09,279.15,278.73,279.00,25056,True,1
604,2025-02-03 15:40:00+00:00,279.22,279.22,279.22,279.22,25500,True,1
605,2025-02-03 16:00:00+00:00,279.22,279.22,278.93,278.99,5082,True,1
